In [33]:
import pandas as pd
import os
import subprocess as sb
import numpy as np

from NORDic.NORDic_PMR.functions import greedy

In [34]:
file_folder="./" 
save_folder=f"{file_folder}/PMR/" 
seed_number = 12345
njobs = 9

if (not os.path.exists(save_folder)):
    proc = sb.Popen(f"mkdir -p {save_folder}".split(" "))
    proc.wait()

In [35]:
gene_list_path=f"{file_folder}only_genes/solution.bnet"
with open(gene_list_path, "r") as file:
    genes = [g.split(",")[0] for g in file.read().split("\n")]
genes = [g for g in list(set(genes)) if (len(g)!=0)] ## remove duplicates and "empty" genes
gene_outputs = genes
genes[:10]

['HGF',
 'TGFB1',
 'COL6A3',
 'SMPD1',
 'EGLN1',
 'TIMP1',
 'POLR2F',
 'LTA',
 'PSME2',
 'GRPEL1']

In [13]:
# Patient state (PTSD) from Breen et al., Nature 2015 (DOI: 10.1038/mp.2015.9)
states = pd.read_csv("Marines_post_PTSD.csv")
states = states.set_index("GeneSymbols")
states

,Sample_1_3,Sample_3_3,Sample_5_3,Sample_7_3,Sample_9_3,Sample_11_3,Sample_13_3,Sample_15_3,Sample_17_3,Sample_19_3,Sample_21_3,Sample_23_3,Sample_25_3,Sample_27_3,Sample_29_3,Sample_31_3,Sample_33_3,Sample_35_3,Sample_37_3,Sample_39_3,Sample_41_3,Sample_43_3,Sample_45_3,Sample_47_3,Sample_49_3,Sample_51_3,Sample_53_3,Sample_55_3,Sample_57_3,Sample_59_3,Sample_61_3,Sample_63_3,Sample_65_3,Sample_67_3,Sample_69_3,Sample_71_3,Sample_73_3,Sample_75_3,Sample_77_3,Sample_79_3,Sample_81_3,Sample_83_3,Sample_85_3,Sample_87_3,Sample_89_3,Sample_91_3,Sample_93_3
GeneSymbols,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A1BG,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
AAAS,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
AACS,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
AAGAB,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
AAK1,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
ZXDB,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
ZYG11B,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
# ## Generate at random initial/patients states
# npatients=17
# prop_nan=0.
# states = pd.DataFrame(
#     np.random.choice([0,1,-1], p=[(1-prop_nan)/2, (1-prop_nan)/2, prop_nan], size=(len(genes), npatients)), 
#                       index=genes, 
#                       columns=[f"state{i+1}" for i in range(npatients)], dtype=object)
# states = states.loc[~states.index.duplicated()] # remove duplicated genes
# states = states.loc[[g for g in states.index if (len(g)!=0)]] # remove genes with no name
# states=states.loc[[g for g in states.index if g in genes]] # keep only genes in network
# states[states==-1] = np.nan

In [36]:
## Parameters
IM_params = {
    "seed" : seed_number, "gene_inputs" : genes,
    "gene_outputs" : gene_outputs
}
SIMU_params = {
    'nb_sims': 100, 'rates': "fully_asynchronous", 'thread_count': njobs,
    'depth': "constant_unitary",
}
k=1 ## number of simultaneous gene perturbations

In [37]:
S, spreads = greedy(f"{file_folder}only_genes/solution.bnet", k, states, IM_params, SIMU_params, 
                    save_folder=save_folder)     

In [44]:
#spreads.sort_values(by=spreads.columns[0], ascending = False)

In [38]:
values=spreads[[spreads.columns[0]]].sort_values(by=spreads.columns[0], ascending=False)
values

,[['EZR']]
EZR,0.163452
VEGFA,0.144073
IL6,0.140864
SERPINE1,0.132246
TNF,0.126289
HGF,0.123047
F9,0.119065
PROC,0.118931
CDKN1A,0.118594
TNFRSF9,0.117778


In [39]:
# Top 3 regulators genes
values[:3]

,[['EZR']]
EZR,0.163452
VEGFA,0.144073
IL6,0.140864


In [9]:
# Network export in cytoscape format
from NORDic.NORDic_NI.functions import solution2cytoscape
from NORDic.UTILS.utils_grn import load_grn

file_folder = "./only_experiments/"
solution = load_grn(file_folder + "solution.bnet")
solution2cytoscape(solution, file_folder+"solution_cytoscape_jobim")